In [ ]:
!pip install datasets

In [ ]:
import pandas as pd
from datasets import load_dataset
import openai
import string
import re
import anthropic

In [ ]:
client = anthropic.Anthropic(
    api_key="",
)

In [ ]:
# syntreat=pd.read_csv('/content/AROMA/TimeRate_Synthetic_Qs.csv_400.csv')
# syntreat=pd.read_csv('/content/AROMA/Rate_Change_Synthetic_Qs_400.csv')
# syntreat=pd.read_csv('/content/AROMA/Temporal_Qs_400.csv')
# syntreat=pd.read_csv('/content/AROMA/Unit_Conversion_Qs_400.csv')




In [ ]:
data=syntreat

In [ ]:
data['label'] = data['Answer'].apply(lambda response: response.split('####')[-1].strip() if '####' in response else '')


In [ ]:
data.rename(columns={'Question': 'question'}, inplace=True)
test_data=data[['question', 'label']]

In [ ]:
test_data = test_data[test_data['label'].apply(lambda x: str(x).isdigit())]

In [ ]:
test_data=test_data[:371]

In [ ]:
class OpenAIChat:
    def __init__(self, model="claude-instant-1.2"):
    # def __init__(self, model="claude-2.0"):
    # def __init__(self, model="claude-2.1"):
    # def __init__(self, model="claude-3-haiku-20240307"):
        self.model = model

    def ask_openai(self, prompt_text):
        """Send a prompt to the OpenAI Chat API and return the response."""
        try:
            response = client.messages.create(
                model=self.model,
                messages=[
                    {"role": "user", "content": prompt_text}
                ],
                temperature=1,
                max_tokens=1000,
                top_p=1,

            )
            if response.content and len(response.content) > 0:
                content_text = response.content[0].text.strip()
                return content_text
            else:
                print("No content found in response.")
                return None

        except Exception as e:
            print(f"An error occurred: {e}")
            return None

    def create_prompt(self, question):
        return (f"Your task is to consider the problem presented below and provide a detailed explanation of your "
                "reasoning process, step by step. At the end of your explanation, you must clearly indicate the final "
                "numerical answer. Round your answers to the nearest whole number."
                "To do this, use the phrase 'Answer:' followed directly by the numerical answer, "
                "with no spaces in between. Ensure that the numerical answer is the only content following 'Answer:'. "
                "Do not enclose the answer within any symbols or brackets, and avoid including any additional text or "
                "characters after the answer."
                "Here is the problem for you to solve:\n\n"
                f"Question: {question}")


    def extract_numerical_answer(self, response):
        """Extract a numerical answer from the response, considering spaces and units."""
        pattern = r'Answer:\s*(\$|€|£)?\s*([\d,]+(?:\.\d+)?)\s*([a-zA-Z%$]+)?'
        match = re.search(pattern, response)
        if match:
            currency_symbol = match.group(1) if match.group(1) else ""
            number = match.group(2).replace(',', '')  # Remove commas from the number
            unit = match.group(3) if match.group(3) else ""
            answer = f"{currency_symbol}{number}{unit}".strip()
            return answer
        else:
            return "Answer not found or not in expected format."

    def ensure_same_type(self, model_answer, true_answer):
        """Ensure model answer and true answer are of the same type for comparison,
        handling currency symbols and units."""

        # Function to sanitize and convert answer to float if possible
        def convert_to_number(answer):
            # Remove currency symbols (and potentially other non-numeric characters except period and minus sign)
            sanitized_answer = re.sub(r'[^\d.-]', '', answer)
            try:
                # Attempt to convert to float
                return float(sanitized_answer)
            except ValueError:
                return None

        model_answer_num = convert_to_number(model_answer)
        true_answer_num = convert_to_number(true_answer)

        # Check if either answer could not be converted to float
        if model_answer_num is None or true_answer_num is None:
            print(f"Conversion error: could not convert to number. Model: {model_answer}, True: {true_answer}")
            return None, None

        # Optionally, convert to int if the number is a whole number
        if model_answer_num.is_integer() and true_answer_num.is_integer():
            return int(model_answer_num), int(true_answer_num)

        return model_answer_num, true_answer_num

    def evaluate_answer(self, question, true_answer):
        detailed_prompt = self.create_prompt(question)
        print('detailed_prompt', detailed_prompt)
        model_answer = "Answer not found or not in expected format."
        retry_count = 0  # Initialize a counter for retries

        while model_answer == "Answer not found or not in expected format." and retry_count < 10:
            response = self.ask_openai(detailed_prompt)
            print('response', response)
            if response:
                # Try to extract the numerical answer from the response
                temp_model_answer = self.extract_numerical_answer(response)

                if temp_model_answer != "Answer not found or not in expected format.":
                    model_answer = temp_model_answer  # Update model_answer with the newly extracted value
                else:
                    # If the extracted answer is not in the correct format, loop will continue
                    print("Retrying due to incorrect format...")
                    retry_count += 1
            else:
                return {
                    "question": question,
                    "error": "Failed to get response from OpenAI."
                }
            # Check if we've exceeded the retry limit
        if retry_count >= 10:
            print("Reached maximum retry attempts. Moving to the next question.")
            return {
                "question": question,
                "error": "Answer not found or not in expected format after retries."
            }

        # Once a valid model_answer is obtained, proceed with type checking and comparison
        model_answer, true_answer = self.ensure_same_type(model_answer, str(true_answer))
        is_correct = model_answer == true_answer
        print('model_answer', model_answer)
        print('true_answer', true_answer)
        print('is_correct', is_correct)
        return {
            "question": question,
            "model_answer": model_answer,
            "true_answer": true_answer,
            "is_correct": is_correct,
            "response": response
        }

def evaluate_dataset(model_evaluator, dataset):
    results = []
    for index, row in dataset.iterrows():
        question = row['question']
        true_answer = row['label']

        result = model_evaluator.evaluate_answer(question, true_answer)
        results.append(result)

        # Optionally, print interim results every 100 iterations
        if (index + 1) % 100 == 0:
            print(f'Interim evaluation completed for {index + 1} entries.')

    print('Evaluation completed for all entries.')
    return results

chat_evaluator = OpenAIChat()

# Assuming `test_data` is your dataset for testing
evaluation_results = evaluate_dataset(chat_evaluator, test_data)


In [ ]:
evaluation_df = pd.DataFrame(evaluation_results)

In [ ]:
is_correct_counts = evaluation_df['is_correct'].value_counts()

# Print the value counts
print(is_correct_counts)

is_correct
True     113
False     77
Name: count, dtype: int64
